## Criando uma rede neural convolucional
***

Neste tutorial iremos criar uma rede neural convolucional para detecção de gatos e cachorros

***

In [1]:
from keras.models import Sequential, model_from_json
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.callbacks.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np

Using TensorFlow backend.


In [2]:
classificador = Sequential()

In [3]:
# Saída é uma matriz de caracteristicas
# Imagem (h * w * d) = (64 * 64 * 3) = 12.288 neuronios
# Filtro (fh * fw * fd) = (3 * 3 * 1) = 9 neuronios
# Matriz de caracteristicas (h - fh + 1) x (w - fw + 1) x 1 = (64 - 3 + 1) x (64 - 3 + 1) = 62 x 62 = 3.844 neuronios
dimensoes_do_kernel = (3, 3) # matriz 3x3
dimensoes_da_imagem = (64, 64, 3) # (largura, altura, canais rgb)
classificador.add(Conv2D(
    filters = 32,
    kernel_size = dimensoes_do_kernel,
    input_shape = dimensoes_da_imagem,
    activation = 'relu'
))

In [4]:
# Vai acelerar o processamento, ou seja, vai pegar o mapa de caracteristicas
# e vai normaliza os valores em uma escala entre 0 e 1
classificador.add(BatchNormalization())

In [5]:
# Usado para pegar o maior valor do mapa de caracteristicas, realçando-as.
# Matriz de caracteristica (h x w x d) = 62 x 62 x 1 = 3.844 neuronios
# Pooling Matrix (ph x pw x pd) = 2 x 2 x 1 = 4
# Matriz de características realçadas (h/ph x w/pw x d/pd) = 31 x 31 x 1 = 961 neuronios
matriz_de_pooling = (2, 2) # matriz 2x2
classificador.add(MaxPooling2D(pool_size = matriz_de_pooling))

In [6]:
# Adicionando outra camada de convolução
classificador.add(Conv2D(
    filters = 32,
    kernel_size = dimensoes_do_kernel,
    input_shape = dimensoes_da_imagem,
    activation = 'relu'
))
classificador.add(BatchNormalization())
classificador.add(MaxPooling2D(pool_size = matriz_de_pooling))

In [7]:
# Adicionar o flattening para vetorizar a matriz para entrar na rede neural densa
classificador.add(Flatten())

In [8]:
# Adicionar a primeira camada da rede neural densa (camada de entrada).
# UNITS = Quantidade de neuronios que farão parte da camada ((neuronios + 1)/2)
# ACTIVATION = Função de ativação
# KERNEL_INITIALIZER = Como você vai fazer a inicialização dos pesos
classificador.add(Dense(
    units = int((961+1)/2),
    activation = 'relu'
))

In [9]:
# Vamos zerar 20% dos valores de entrada para evitar o overfitting
classificador.add(Dropout(0.2))

In [10]:
# Vamos criar a primeira camada oculta da rede neural densa
classificador.add(Dense(
    units = int((961+1)/2),
    activation = 'relu'
))

In [11]:
# Vamos zerar 20% dos valores de entrada para evitar o overfitting
classificador.add(Dropout(0.2))

In [12]:
# Vamos criar a camada de saída da rede neural densa
classificador.add(Dense(
    units = 1,
    activation = 'sigmoid'
))

In [13]:
# Vamos compilar a rede neural
classificador.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

***

In [14]:
# Cria novas imagens a partir das imagens existentes
normalizacao = 1./255
# ROTATION_RANGE = Grau de rotação da imagem
# HORIZONTAL_FLIP = Vai fazer giros horizontais nas imagens
# SHEAR_RANGE = Faz a mudança dos pixels para outra direção
# HEIGHT_SHIFT_RANGE = Faixa de mudança da altura da imagem
# ZOOM_RANGE = Faixa de mudança do zoom da imagem
gerador_de_imagens = ImageDataGenerator(
    rescale = normalizacao,
    rotation_range = 7,
    horizontal_flip = True,
    shear_range = 0.2,
    height_shift_range = 0.07,
    zoom_range = 0.2
)

In [15]:
# Cria novas imagens de teste a partir das imagens existentes
gerador_de_imagens_teste = ImageDataGenerator(rescale = normalizacao)

In [16]:
# Criar a base de dados de treinamento
# TARGET_SIZE = Tamanho das imagens (altura, largura)
# BATCH_SIZE = De quantas em quantas imagens será feito o treinamento (de 125 em 125 no caso) - 4000/32 = 125
# CLASS_MODE = Tipo de saída, no caso é binario
base_treinamento = gerador_de_imagens.flow_from_directory(
    'dataset/training_set',
    target_size = (64, 64),
    batch_size = 32,
    class_mode = 'binary'
)

Found 4000 images belonging to 2 classes.


In [17]:
 # Criar a base de dados de teste
# BATCH_SIZE = De quantas em quantas imagens será feito o teste (de 31 em 31 no caso) - 1000/32 = 31
base_teste = gerador_de_imagens_teste.flow_from_directory(
    'dataset/test_set',
    target_size = (64, 64),
    batch_size = 32,
    class_mode = 'binary'
)

Found 1000 images belonging to 2 classes.


In [18]:
# Irá salvar o modelo a cada ciclo (epoch) e armazenar o melhor
# FILEPATH = Nome do modelo salvo a cada ciclo
# MONITOR = Atributo que será monitorado
# SAVE_BEST_ONLY = Salva somente os melhores resultados, não os deixando ser sobrescritos por outros
# SAVE_WEIGHTS_ONLY = Salvar somente os pesos do modelo (model.save_weights(filepath))
# VERBOSE = Aparecer na tela feedbacks sobre o checkpoint
checkpointer = ModelCheckpoint(
    filepath="melhores_pesos.h5",
    monitor="val_loss",
    verbose=1,
    save_best_only=True,
    save_weights_only=True
)

In [19]:
# Reduzir a taxa de treinamento quando as métricas pararem de subir (evoluir)
# MONITOR = Atributo que será monitorado no treinamento
# FACTOR = Fator que irá reduzir a taxa de aprendizagem (new_lr = lr * factor)
# PATIENCE = Se ocorrer 2 vezes a não evolução da métricas reduzir o lr
# MIN_LR = Menor taxa de aprendizado que deve chegar
# VERBOSE = Aparecer na tela feedbacks sobre a redução da taxa de aprendizado
reduce_learning_rate = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.2,
    verbose=1,
    patience=2,
    min_lr=0.001
)

In [20]:
# Para o treinamento quando o mesmo não evolui mais
# MONITOR = Atributo que será monitorado no treinamento
# PATIENCE = Quantidade de vezes que deve ocorrer a não evolução até parar
# RESTORE_BEST_WEIGHTS = Restaura os melhores pesos que ocorreram no treinamento.
# VERBOSE = Aparecer na tela feedbacks sobre a parada do treinamento
early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=3,
    verbose=1,
    restore_best_weights=True
)

In [21]:
# Fazer o treinamento
# STEPS_PER_EPOCH = Quantidade de imagens que será treinada por ciclo, ideal o total de imagens
# EPOCHS = Quantidade de ciclos que será treinado, quanto maior melhor
# VALIDATION_DATA = Imagens de teste
# VALIDATION_STEPS = Quantidade de imagens de teste por ciclo
classificador.fit_generator(
    base_treinamento,
    steps_per_epoch = 4000/32,
    epochs = 5,
    validation_data = base_teste,
    validation_steps = 1000/32,
    callbacks=[checkpointer, reduce_learning_rate, early_stopping]
)

Epoch 1/5
125/125 [==============================] - 63s 504ms/step - loss: 0.8629 - accuracy: 0.5878 - val_loss: 3.3054 - val_accuracy: 0.5000

Epoch 00001: val_loss improved from inf to 3.30539, saving model to melhores_pesos.h5
Epoch 2/5
125/125 [==============================] - 80s 641ms/step - loss: 0.6558 - accuracy: 0.6442 - val_loss: 1.2180 - val_accuracy: 0.5000

Epoch 00002: val_loss improved from 3.30539 to 1.21805, saving model to melhores_pesos.h5
Epoch 3/5
125/125 [==============================] - 75s 599ms/step - loss: 0.6131 - accuracy: 0.6695 - val_loss: 2.7239 - val_accuracy: 0.5170

Epoch 00003: val_loss did not improve from 1.21805
Epoch 4/5
125/125 [==============================] - 59s 472ms/step - loss: 0.5875 - accuracy: 0.6945 - val_loss: 0.4869 - val_accuracy: 0.5530

Epoch 00004: val_loss improved from 1.21805 to 0.48694, saving model to melhores_pesos.h5
Epoch 5/5
125/125 [==============================] - 53s 421ms/step - loss: 0.5490 - accuracy: 0.7272 -

In [22]:
# Salvando a estrutura da rede neural
classificador_json = classificador.to_json()
with open('classificador.json', 'w') as json_file:
    json_file.write(classificador_json)

In [23]:
# Salvando os pesos da rede neural (pip install h5py)
classificador.save_weights('pesos.h5')

***

In [24]:
# Pegando a estrutura da rede neural
with open('classificador.json', 'r') as json_file:
    estrutura = json_file.read()

In [25]:
# Criando o classificador
classificador_externo = model_from_json(estrutura)

In [26]:
# Pegando os pesos da rede neural
classificador_externo.load_weights('melhores_pesos.h5')

In [27]:
imagem_teste = image.load_img('dataset/test_set/gato/cat.3500.jpg', target_size = (64, 64))

In [28]:
imagem_teste_array = image.img_to_array(imagem_teste)

In [29]:
# Normalização
imagem_teste_array /= 255

In [30]:
# Formatar para o formato do tensorFlow (qtd, largura, altura, canais)
imagem_teste_array = np.expand_dims(imagem_teste_array, axis = 0)

In [31]:
previsao = classificador_externo.predict(imagem_teste_array)

In [32]:
print(base_treinamento.class_indices)

{'cachorro': 0, 'gato': 1}


In [33]:
# 0 < previsao < 0.5 = Cachorro
# 0.5 <= previsao < 1 = Gato
print(previsao)
if previsao >= 0.5:
    print("É um gato")
else:
    print("É um cachorro")

[[0.47291747]]
É um cachorro
